<a href="https://colab.research.google.com/github/HughClaimsend/MA5851_HughMcMullan_A3/blob/main/MA5851_HughMcMullan_A3_Part1_Scraping_Drone_Data_with_Selenium_and_BeautifulSoup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Notebook for web-scraping done posts, using Selenium and Beautiful Soup**

In [4]:
#Importing Selenium packages for crawling and scraping web pages
!pip install selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 https://developer.download.nvidia.com/compute/c

In [2]:
#Import other libraries
from bs4 import BeautifulSoup     # BeautifulSoup for parsing scraped pages
import re                         # Regex functions
import csv                        # Writing to csv file
import datetime                   # Generating timestamps

In [5]:
# Define Selenium driver using Chrome libraries
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: use options instead of chrome_options
  


In [6]:
# function to fetch and scrape a website page
def fetch_and_scrape_website(url):
  # Fetch the web page [with Selenium]
  print("\nFETCHING ...", url)
  driver.get(url)       

  # Selenium hands the page source to Beautiful Soup
  soup = BeautifulSoup(driver.page_source, 'lxml')

  # Closes all browser windows and safely ends the session
  #driver.close()

  # return the soup object
  return(soup)

In [7]:
def strip_thread(thread):
  rtn = thread
  if thread[-12:-7] == "post-":
    rtn = thread[:-12]
  return(rtn)

In [8]:
# function to scrape a set of SEARCH RESULT pages using a page_no
def scrape_and_save_search_pages(dronesite_details_list, max_pages, max_items, output_filename):
  dronesite_name = dronesite_details_list[0]
  url_base = dronesite_details_list[1]
  search_identifier = dronesite_details_list[2]
  dronesite_group = dronesite_details_list[3]
  print(dronesite_name)
  print(dronesite_group)
  print(url_base)
  print(search_identifier)
  
  # syntax for mavic/phantom etc websites
  search_url_base = url_base + "/search/" + search_identifier + "/?q=casa&o=relevance"
  print("Search thread:", search_url_base)
  print("\n")

  for page in range(max_pages):    
    url_page = search_url_base + "&page=" + str(page + 1)   # incorporate search page no 
    soup = fetch_and_scrape_website(url_page)               # load the page into soup
    
    # Run through all the threads found by the search
    threads = soup.find_all('h3',{'class':'contentRow-title'})
    items_count = 0
    for thread in threads:
      href = thread.find("a")["href"]
      thread_name = thread.find("a").text

      # Print this block so scraping can be monitored
      print("\n=====================================================================")
      print("Search thread:", thread_name)
      print("=====================================================================")

      thread_url = url_base + strip_thread(href)

      scrape_and_save_thread_pages(dronesite_name, dronesite_group, thread_name, thread_url, max_items, output_filename)

      items_count += 1
      if items_count > max_items:
          break


In [9]:
# function to scrape the FIRST THREAD page (even if there are multiple pages)
def scrape_and_save_thread_pages(dronesite_name, dronesite_group, thread_name, thread_url, max_items, output_filename):
  thread_no = thread_url[-7:]
  thread_no = thread_no[:-1]
  print(thread_no, thread_name, thread_url)

  soup = fetch_and_scrape_website(thread_url)               # load the page into soup
    
  # Run through all the threads found by the search
  posts = soup.find_all('div',{'class':'message-main'})
  items_count = 0
  for post in posts:
    # Find and extract post data from the thread html
    post_datetime = post.find('time')["datetime"]
    post_article = post.find('article')
    post_text = post_article.find('div',{'class':'bbWrapper'})
    post_text = post_text.text
    print("\n--------------------------------------------------------------------")
    print("Posted at:", post_datetime)
    print("--------------------------------------------------------------------")
    print("Post text:", post_text)

    # Save the posts data
    list_to_save = [dronesite_name, dronesite_group, thread_name, thread_no, post_datetime, post_text]
    save_post_to_csv(output_filename, list_to_save)

    # Stop the loop if max no of post items is reached
    items_count += 1
    if items_count > max_items:
        break

In [10]:
# Write the passed-in list of post details to a line in the .csv file
def save_post_to_csv(output_filename, list_input):
  try:
    with open(output_filename, "a" ) as fopen:    # Open the csv file
      csv_writer = csv.writer(fopen)
      csv_writer.writerow(list_input)
  except:
      return False


In [ ]:
# Set a list website targets for posts scraping
drone_website_targets = [
                         ["MavicPilots", "https://mavicpilots.com", "707065", "Private"],
                         ["PhantomPilots", "https://phantompilots.com", "213313", "Private"],
                         ["FPVDronePilots", "https://fpvdronepilots.com", "15671", "Private"],
                         ["CommercialDronePilots", "https://commercialdronepilots.com", "18113", "Commercial"],
                         ["MatricePilots", "https://matricepilots.com", "49374", "Commercial" ],
                         ["InspirePilots", "https://inspirepilots.com", "56174", "Commercial"]
]

# Set arbitrary limits to avoid too many calls to website(s)
max_page_calls = 10
max_items_per_page = 20

# Initialise dronesite_output file
datetime_now = datetime.datetime.now()
timestamp = datetime_now.strftime("%Y%m%d%H%M")
dronesite_output_filename = "DronePosts_" + timestamp + ".csv"
#print(dronesite_output_filename)

**TESTING ONLY - 1 WEBSITE ONLY, MAXIMUM 2 SEARCH PAGES, MAXIMUM 4 THREADS PER SEARCH PAGE**

In [11]:
# FOR TESTING
# Set a list website targets for posts scraping
drone_website_targets = [
                         ["MavicPilots", "https://mavicpilots.com", "707065", "Private"]
]

# Set arbitrary limits to avoid too many calls to website(s)
max_page_calls = 2
max_items_per_page = 4

# Initialise dronesite_output file
datetime_now = datetime.datetime.now()
timestamp = datetime_now.strftime("%Y%m%d%H%M")
dronesite_output_filename = "DronePosts_" + timestamp + ".csv"
#print(dronesite_output_filename)

In [12]:
# Run through the 'CASA' Search pages for each target website
for dronesite_details_list in drone_website_targets:
  scrape_and_save_search_pages(dronesite_details_list, max_page_calls, max_items_per_page, dronesite_output_filename)


MavicPilots
Private
https://mavicpilots.com
707065
Search thread: https://mavicpilots.com/search/707065/?q=casa&o=relevance



FETCHING ... https://mavicpilots.com/search/707065/?q=casa&o=relevance&page=1

Search thread: Latest CASA newsletter re VLOS
117004 Latest CASA newsletter re VLOS https://mavicpilots.com/threads/latest-casa-newsletter-re-vlos.117004/

FETCHING ... https://mavicpilots.com/threads/latest-casa-newsletter-re-vlos.117004/

--------------------------------------------------------------------
Posted at: 2021-10-07T19:15:52-0400
--------------------------------------------------------------------
Post text: ..
Latest CASA newsletter has VLOS article where you can only do this watching your drone by sight at all times when flying.
Not via goggles (understandable), not by phone, ipad, or tablet, and not by other monitor device.

Not sure how many fly drones there at CASA (I know many do), but how are you to film / photograph, without looking mainly at your device screen 